In [1]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [4]:
df = pd.read_csv("100k_removed_num_and_bangla.csv")
df.head()
# remove duplicate rows
df = df.drop_duplicates(subset=['sentence'])
df.head()

,e_index,sentence,clean_validated_ipa
0,12,গৃহ নিজ কার্য সমাপল জান।।কো সখী দধি মন্থন করু ...,gɾɪho nɪɟ kɐɾɟo ʃɔmɐpol ɟɐn।।ko ʃokʰɪ d̪od̪ʱɪ ...
2,69,থাইল্যান্ড বা শ্যামদেশের রাজধানী।তপন এক মাস হল...,t̪ʰɐɪ̯lɛnd bɐ ʃɛmd̪eʃeɾ ɾɐɟd̪ʱɐnɪ।t̪ɔpon ɛk mɐ...
3,66,শিক্ষে হোক বেটার।দেব-বাড়ির ডাকাতি ও বংশীর ছেল...,ʃɪkkʰe hok betɐɾ।d̪eb-bɐɽɪɾ dɐkɐt̪ɪ o bɔŋʃɪɾ c...
4,30,"যদি বলেন, বাড়িতে নেই!খুব হৈ-হল্লা করলে শেষ পর্...","ɟod̪ɪ bɔlen, bɐɽɪt̪e neɪ̯!kʰub ho͡ɪ̯-hollɐ koɾ..."
5,66,"বাঃ, বেশ নাম তো তোমার।কেমন ইতিহাসের গন্ধ আছে, ...","bɐ, beʃ nɐm t̪o t̪omɐɾ।kɛmon ɪt̪ɪhɐʃeɾ gɔnd̪ʱo..."


In [7]:
# char_list_dependent=['̯', 'ʰ', 'ʱ', 'ʰ', '̪', '̪', '̃', '̪', 'ʰ',"ʲ",'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̑', '̪', '̯', '͡',]
char_list_unique_all=['F', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'z', 'ø', 'ŋ', 'ɐ', 'ɔ', 'ɛ', 'ɟ', 'ɡ', 'ɦ', 'ɪ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̑', '̪', '̯', '͡']

char_list_unique=['F','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'z', 'ø', 'ŋ', 'ɐ', 'ɔ', 'ɛ', 'ɟ', 'ɡ', 'ɦ', 'ɪ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̑', '̪', '̯', '͡',]

dependent_ipa_list=["ʰ","̃","ʱ","","̯","̪","̱","̊","̚","̈","̥","̬","̤","̰","̼","̹","̜","̟","̠","̽","̩","̯","̮","̪","̤","̞","̝","̘","̙","̆","̑","̊","̌",""]

ipa_vowels=['a', 'e', 'i', 'o', 'u', 'ɐ', 'ɔ', 'ɛ', 'ɪ']
ipa_dependent=['̃', 'ː', '̯', '̑', 'ʰ', 'ʱ', '̪', 'ʷ', '͡', 'ʲ', '̆']
ipa_short_vowels=['ɪ', 'ɛ', 'æ', 'ɑ', 'ɔ', 'ʊ', 'ʌ', 'ə']
ipa_extra=['õ']

ipa_consonants=['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'r', 's', 't', 'v', 'z', 'ŋ', 'ɟ', 'ɡ', 'ɦ', 'ɽ', 'ɾ', 'ʃ']
# ipa_consonants=['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'r', 's', 't', 'v', 'z', 'ø', 'ŋ', 'ɟ', 'ɡ', 'ɦ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ']


In [8]:

for row in df.itertuples():
  if type(row.clean_validated_ipa)==float:
    continue
  for ch in row.clean_validated_ipa:
    if ch not in char_list_unique_all:
      char_list_unique_all.append(ch)

print(char_list_unique_all)

In [10]:
  
char_list_unique_all.sort()
print(char_list_unique_all)

[' ', '!', '%', "'", '(', ')', '*', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '~', '\xa0', '\xad', '·', 'ø', 'ŋ', 'ɐ', 'ɔ', 'ɛ', 'ɟ', 'ɡ', 'ɦ', 'ɪ', 'ɽ', 'ɾ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', 'ː', '̃', '̆', '̑', '̪', '̯', '͡', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', '৷', '\u200c', '–', '—', '‖', '‘', '’', '“', '”', '•', '…', '\ufeff']


In [25]:
test="ɔʃɔhɐʲot̪t̪o"
for ch in test:
  print(ch)

ɔ
ʃ
ɔ
h
ɐ
ʲ
o
t
̪
t
̪
o


In [12]:
dependent_ipa="u̯gʰbʱɽʰt̪t̪õd̪ʰ"
list_of_char=[]
for ch in dependent_ipa:

  list_of_char.append(ch)

print(list_of_char)


['u', '̯', 'g', 'ʰ', 'b', 'ʱ', 'ɽ', 'ʰ', 't', '̪', 't', '̪', 'o', '̃', 'd', '̪', 'ʰ']


In [13]:
list_of_char.remove("d")
print(list_of_char)

['u', '̯', 'g', 'ʰ', 'b', 'ʱ', 'ɽ', 'ʰ', 't', '̪', 't', '̪', 'o', '̃', '̪', 'ʰ']
